In [1]:
import numpy as np
import cplex
import random
import time
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
class Subproblem_LP():
    def __init__(self):
        self.n_dim = (n_stage - 1) * n_job * n_job
        self.n_constraints = 2 * (n_stage - 1) * n_job
        self.variable_names = ['x'+ str(i) for i in range(self.n_dim)]
        self.constraints_rows = self.set_constraints_byrows()
        self.opt_solution = np.random.randint(0,2,size = self.n_dim) 
        self.rhs = [1] * self.n_constraints
        self.senses = 'E'* self.n_constraints

    def set_job_to_position(self):
        job_to_position_onlyone = []
        for i in range((n_stage - 1) * n_job):
            job_to_position_onlyone.append([self.variable_names[i * n_job : (i + 1) * n_job], [1] * n_job])
        return job_to_position_onlyone
    
    def set_position_to_job(self):
        position_to_job_onlyone = []
        for j in range(n_stage - 1):
            for i in range(n_job):
                position_to_job_onlyone.append([self.variable_names[j * (n_job**2) + i: (j + 1) * (n_job**2): n_job], [1] * n_job])
        return position_to_job_onlyone
    
    def set_constraints_byrows(self):
        job_to_position_onlyone = self.set_job_to_position()
        position_to_job_onlyone = self.set_position_to_job()
        return job_to_position_onlyone + position_to_job_onlyone
    
    def compute_obj(self, tval, lamd):
        x_coeff_f4 = self.compute_f4_x(lamd)
        x_coeff_F6 = self.compute_F6_x(tval, lamd)
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage-1):
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        for i1 in range(n_job):
                            t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            x_coeff[t2] = x_coeff_f4[t2] - x_coeff_F6[t2]
        self.obj_coeff = x_coeff
        return self.obj_coeff
    
    def compute_F6_x(self, tval, lamd):
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if i1 != i:
                                if r == n_batch * pcast - 1:
                                    t3 = i1 + i * n_job + j * n_job * n_job
                                    t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                                    t5 = i1 + j * n_job
                                    x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t1] -  tval[t5]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) - 0.5 * (2 * processtime[t5] + 1) * (self.opt_solution[t4] - self.opt_solution[t2]))           
                                elif r == 0:
                                    t3 = i + i1 * n_job + j * n_job * n_job
                                    t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                    t5 = i1 + j * n_job
                                    x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t5] -  tval[t1]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) + 0.5 * (2 * processtime[t1] + 1) * (self.opt_solution[t2] - self.opt_solution[t4]))                                       
                                else:
                                    t3 = i + i1 * n_job + j * n_job * n_job
                                    t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                    t5 = i1 + j * n_job
                                    t6 = i1 + i * n_job + j * n_job * n_job
                                    t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                    x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t5] -  tval[t1]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) + 0.5 * (2 * processtime[t1] + 1) * (self.opt_solution[t2] - self.opt_solution[t4]))
                                    x_coeff[t2] += lamd[t6] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t7])**2 - self.opt_solution[t2] - self.opt_solution[t7]) + tval[t1] -  tval[t5]) * (self.opt_solution[t2] + self.opt_solution[t7] - 0.5) - 0.5 * (2 * processtime[t5] + 1) * (self.opt_solution[t7] - self.opt_solution[t2]))           
        
        return x_coeff
    
    def compute_f4_x(self, lamd):
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if i1 != i:
                                if r == n_batch * pcast - 1 or r == 0:
                                    t3 = i + i1 * n_job + j * n_job * n_job
                                    x_coeff[t2] += 0.25 * (2 * processtime[t1] + 1) * lamd[t3]
                                else:
                                    t3 = i + i1 * n_job + j * n_job * n_job
                                    t4 = i1 + i * n_job + j * n_job * n_job
                                    x_coeff[t2] += 0.25 * (2 * processtime[t1] + 1) * (lamd[t3] + lamd[t4])
        return x_coeff

    def set_method(self, method):
        alg = self.subproblem_cplex.parameters.lpmethod.values

        if method == "o":  
            self.subproblem_cplex.parameters.lpmethod.set(alg.auto)
        elif method == "p":
            self.subproblem_cplex.parameters.lpmethod.set(alg.primal)
        elif method == "d":
            self.subproblem_cplex.parameters.lpmethod.set(alg.dual)
        elif method == "b":
            self.subproblem_cplex.parameters.lpmethod.set(alg.barrier)
            self.subproblem_cplex.parameters.barrier.crossover.set(self.Subproblem_LP_TU.parameters.parameters.barrier.crossover.values.none)
        elif method == "h":
            self.subproblem_cplex.parameters.lpmethod.set(alg.barrier)
        elif method == "s":
            self.subproblem_cplex.parameters.lpmethod.set(alg.sifting)
        elif method == "c":
            self.subproblem_cplex.parameters.lpmethod.set(alg.concurrent)
        else:
            raise ValueError('method must be one of "o", "p", "d", "b", "h", "s" or "c"')
        
    def solve(self, method = 'o'):
        self.subproblem_cplex = cplex.Cplex()
        self.subproblem_cplex.objective.set_sense(self.subproblem_cplex.objective.sense.minimize)
        ub = [1] * self.n_dim
    
        self.subproblem_cplex.variables.add(obj = self.obj_coeff, ub = ub, names = self.variable_names)    
    
        assert(self.n_constraints == len(self.constraints_rows))
        row_names = ['c'+ str(i) for i in range(self.n_constraints)]
        #print("the dimension of x:", self.n_dim, "the number of constraints:", self.n_constraints)
        self.subproblem_cplex.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = row_names)
        self.set_method(method)
        self.subproblem_cplex.solve()
        self.get_solvable_status()
        self.get_optvalues()
        
    def write_model(self):
        self.subproblem_cplex.write('Subproblem_LP.lp')
        
    def get_optvalues(self):
        self.obj_values = self.subproblem_cplex.solution.get_objective_value()
        print ("Solution value:   ",  self.obj_values)
        self.opt_solution = []
        for i, x in enumerate(self.subproblem_cplex.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.subproblem_cplex.solution.get_status()
        if status == self.subproblem_cplex.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.subproblem_cplex.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.subproblem_cplex.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def output_results(self):
        with open('Subproblem_LP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
                f.write('\n')

In [3]:
class Subproblem_QP():
    def __init__(self):
        self.n_dim = n_job * n_stage + 2 * pcast * n_machine['steelmaking']
        self.variable_names = ['t' + str(i)  for i in range(self.n_dim)]
        self.duedate = self.compute_duedate()
        self.constraints_rows, self.rhs, self.n_per_constraints = self.set_constraints_byrows()
        self.senses = self.set_senses()
        self.opt_solution = [random.randint(0,10) for i in range(n_stage * n_job)] + 2 * pcast * n_machine['steelmaking'] * [0]
        self.n_constraints = len(self.constraints_rows)
        self.cost_function_iterations = []
        
    def set_preceding_transfer_constraints(self):
        preceding_transfer_constraints = []
        for i in range(n_job * (n_stage - 1)):
            preceding_transfer_constraints.append([[self.variable_names[i], self.variable_names[i + n_job]], [1, -1]])

        rhs_transittime, rhs_processtime = [], processtime[0: n_job * (n_stage - 1)]
        for i in range(len(transittime)):
            rhs_transittime += [transittime[i]] * n_job
        rhs = [ - processtime[i] - rhs_transittime[i] for i in range(n_job * (n_stage - 1))]
        return preceding_transfer_constraints, rhs, rhs_processtime, rhs_transittime

    def set_process_continuous_constraints(self):
        process_continuous, adjacent_casts = [], []
        rhs_continuous_processtime, rhs_adjacent_casts = [], []
        assert(n_job * (n_stage - 1) % n_batch == 0)
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if (i + 1) % n_batch != 0:
                process_continuous.append([[self.variable_names[i], self.variable_names[i + 1]], [1, -1]])
                rhs_continuous_processtime.append(-processtime[i])
            else:
                if (i + 1) % (n_batch * pcast) != 0:
                    adjacent_casts.append([[self.variable_names[i], self.variable_names[i + 1]], [1, -1]])
                    rhs_adjacent_casts.append(- processtime[i] - su)
        return process_continuous, adjacent_casts, rhs_continuous_processtime, rhs_adjacent_casts 

    def set_additional_constraints(self):
        additional_constraints = []
        index = n_job * n_stage
        index_start = pcast * n_machine['casting']
        assert(n_job * (n_stage - 1) % n_batch == 0 and (index % n_batch == 0))
        batch = 0
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if i % n_batch == 0:
                additional_constraints.append([[self.variable_names[i], self.variable_names[index + batch], self.variable_names[index + batch + index_start]], [1, -1, 1]])
                batch += 1
        return additional_constraints
    
    def set_constraints_byrows(self):
        preceding_transfer_constraints, rhs_preceding_transfer, _, _ = self.set_preceding_transfer_constraints()
        process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts = self.set_process_continuous_constraints()
        additional_constraints = self.set_additional_constraints()
    
        rows = preceding_transfer_constraints + process_continuous_constraints + adjacent_casts_constraints + additional_constraints
        
        n_per_constraints = [len(preceding_transfer_constraints), len(process_continuous_constraints), len(adjacent_casts_constraints), len(additional_constraints)]
        
        rhs = rhs_preceding_transfer + rhs_continuous_processtime + rhs_adjacent_casts + self.duedate
        return rows, rhs, n_per_constraints
    
    def set_senses(self):
        s, senses = 'LELE', ''
        for i in range(len(self.n_per_constraints)):
            senses += s[i] * self.n_per_constraints[i]
        return senses
    
    def compute_duedate(self):
        duedate = [0 for i in range(pcast * n_machine['casting'])]
        duedate_start = [0 for i in range(n_machine['casting'])]
        n_cast = pcast * n_machine['casting']
        for i in range(n_machine['casting']):
            index = 0
            for j in range(n_stage - 1):
                duedate_start[i] += processtime[i * n_cast + index * n_job] + transittime[j]
                index += 1
        #print(duedate_start)
    
        for i in range(n_machine['casting']):
            duedate[i * pcast] = duedate_start[i]
            index_start = n_job * (n_stage - 1) + i * n_cast
            for j in range(1, pcast):
                duedate[i * pcast + j] = duedate[i * pcast + j - 1] + sum(processtime[index_start: index_start + n_batch]) + su
                index_start = index_start + n_batch
        return duedate
    
    def compute_obj_wait_time(self):
        obj_wait_time = np.zeros((n_stage, n_job))
        for i in range(n_stage - 1):
            for j in range(n_job):
                obj_wait_time[i + 1][j], obj_wait_time[i][j] = obj_wait_time[i + 1][j] + coeff[i], obj_wait_time[i][j] - coeff[i]
        obj_wait_time = obj_wait_time.reshape(1, n_stage * n_job)
        return np.hstack((obj_wait_time, np.zeros((1, self.n_dim - n_stage * n_job)))).tolist()[0]

    def compute_obj_casting(self):
        obj_casting = np.hstack((d_coeff['d1'] *  np.ones((1, pcast * n_machine['casting'])), d_coeff['d2'] * np.ones((1, pcast * n_machine['casting']))))
        obj_casting = np.hstack((np.zeros((1, n_job * n_stage)), obj_casting))
        return obj_casting.tolist()[0]
    
    def compute_obj_linear(self, xval, lamd):
        self.obj_linear_wait_time = self.compute_obj_wait_time()
        self.obj_linear_casting = self.compute_obj_casting()
        self.obj_linear_F6_t = self.compute_F6_t(xval, lamd)
        self.obj_linear = [self.obj_linear_wait_time[i] + self.obj_linear_casting[i] - self.obj_linear_F6_t[i] for i in range(self.n_dim)]
        return self.obj_linear
    
    def compute_F6_t(self, xval, lamd):
        t_coeff = [0 for i in range((n_stage - 1) * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for k in range(n_machine_perstage):
                    for r in range(n_batch * pcast - 1):
                        for i1 in range(n_job):
                            if i1 != i:
                                t2 = i + i1 * n_job + j * n_job * n_job
                                t3 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t6 = i1 + i * n_job + j * n_job * n_job
                                t7 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t8 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t_coeff[t1] -= lamd[t2] * (0.5 * ((xval[t3] + xval[t4])**2 - xval[t3] - xval[t4]) + self.opt_solution[t5] -  self.opt_solution[t1]) 
                                t_coeff[t1] += lamd[t6] * (0.5 * ((xval[t7] + xval[t8])**2 - xval[t7] - xval[t8]) + self.opt_solution[t1] -  self.opt_solution[t5])            
        return t_coeff + [0] * (self.n_dim - len(t_coeff)) 
    
    def compute_qmat(self, lamd):
        quad_t_coeff = np.zeros((self.n_dim, self.n_dim))
        for j in range(n_stage - 1):
            for i in range(n_job):
                for i1 in range(n_job):
                    if i1 != i:
                        t1 = i + i1 * n_job + j * n_job * n_job
                        t2 = i1 + i * n_job + j * n_job * n_job 
                        quad_t_coeff[i + j * n_job, i + j * n_job] += (lamd[t1] + lamd[t2]) * n_machine_perstage * (n_batch * pcast - 1)
                for i in range(n_job - 1):
                    for i1 in range(i + 1, n_job):
                        t1 = i + i1 * n_job + j * n_job * n_job
                        t2 = i1 + i * n_job + j * n_job * n_job
                        quad_t_coeff[i + j * n_job, i1 + j * n_job] = -(lamd[t1] + lamd[t2]) * n_machine_perstage * (n_batch * pcast - 1)
                        quad_t_coeff[i1 + j * n_job, i + j * n_job] = -(lamd[t1] + lamd[t2]) * n_machine_perstage * (n_batch * pcast - 1)                            
        quad_t_coeff = quad_t_coeff.tolist()
        self.qmat = [[self.variable_names, quad_t_coeff[i]] for i in range(len(quad_t_coeff))]
        return self.qmat
    
    def solve(self):
        self.subproblem_cplex = cplex.Cplex()
        self.subproblem_cplex.objective.set_sense(self.subproblem_cplex.objective.sense.minimize)
        
        ub  = [cplex.infinity] * self.n_dim
        lb  = [0] * self.n_dim
        self.subproblem_cplex.variables.add(obj = self.obj_linear, ub = ub, lb = lb, names = self.variable_names)
        self.subproblem_cplex.objective.set_quadratic(self.qmat)
    
        rownames = ['c' + str(i)  for i in range(self.n_constraints)]

        self.subproblem_cplex.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = rownames)
        self.subproblem_cplex.solve()
        self.get_solvable_status()
        self.get_optvalues()

    def write_model(self):
        self.subproblem_cplex.write('Subproblem_QP.lp')
        
    def get_optvalues(self):
        self.obj_values = self.subproblem_cplex.solution.get_objective_value()
        print ("Solution value:   ",  self.obj_values)
        self.opt_solution = []
        for i, x in enumerate(self.subproblem_cplex.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
        self.cost_function_iterations.append(self.subproblem_cplex.solution.get_objective_value())
                
    def get_solvable_status(self):
        status = self.subproblem_cplex.solution.get_status()
        if status == self.subproblem_cplex.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.subproblem_cplex.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.subproblem_cplex.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")

    def print_constraints_slack(self):
        slack = self.subproblem_cplex.solution.get_linear_slacks()
        for i in range(len(slack)):
            print("Row %d:  Slack = %10f" % (i, slack[i]))     
            
    def plot_results(self):
        plt.figure()
        x = np.linspace(0, len(self.cost_function_iterations), len(self.cost_function_iterations))
        plt.plot(x, self.cost_function_iterations)
        plt.xlabel('iteration times')
        plt.ylabel('cost function')
        plt.show()
        
    def output_results(self):
        with open('Subproblem_QP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
                f.write('\n')
        with open('obj_linear_F6_t.txt','w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.obj_linear_F6_t[i * n_job:(i + 1) * n_job]))
                f.write('\n')

In [4]:
class List_Schedule(Subproblem_QP):
    def __init__(self, xval):
        self.n_dim = n_job * n_stage + 2 * pcast * n_machine['steelmaking']
        self.variable_names = ['t' + str(i)  for i in range(self.n_dim)]
        self.xval = xval
        self.duedate = self.compute_duedate()
        self.constraints_rows, self.rhs, self.n_per_constraints = self.set_constraints_byrows()
        self.senses = self.set_senses()
        self.n_constraints = len(self.constraints_rows) 
        
    def compute_obj(self):
        t_coeff_w = self.compute_obj_wait_time()
        t_coeff_c = self.compute_obj_casting()
        t_coeff = [t_coeff_w[i] + t_coeff_c[i] for i in range(self.n_dim)]
        return t_coeff
    
    def set_machine_capacity_constraints(self):
        machine_capacity_constraints = []
        rhs_machine_capacity_processtime = []
        for i in range(n_job):
            for i1 in range(n_job):
                if i != i1:
                    for j in range(n_stage - 1):
                        t1 = i + j * n_job
                        t2 = i1 + j * n_job
                        for r in range(n_batch * pcast - 1):
                            for k in range(n_machine_perstage):
                                t3 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1 
                                t5 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                if self.xval[t3] == 1 and self.xval[t4] == 1:
                                    machine_capacity_constraints.append([[self.variable_names[t2], self.variable_names[t1]],[-self.xval[t3] * self.xval[t4], self.xval[t3] * self.xval[t4]]])
                                    rhs_machine_capacity_processtime.append(-self.xval[t3] * self.xval[t4] * processtime[t1]) 
        return machine_capacity_constraints, rhs_machine_capacity_processtime
    
    def set_constraints_byrows(self):
        machine_capacity_constraints, rhs_machine_capacity_processtime = self.set_machine_capacity_constraints()
        preceding_transfer_constraints, rhs_preceding_transfer, _, _ = self.set_preceding_transfer_constraints()
        process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts = self.set_process_continuous_constraints()
        additional_constraints = subproblem_qp.set_additional_constraints()
    
        rows = machine_capacity_constraints + preceding_transfer_constraints + process_continuous_constraints + adjacent_casts_constraints + additional_constraints
        
        n_per_constraints = [len(machine_capacity_constraints), len(preceding_transfer_constraints), len(process_continuous_constraints), len(adjacent_casts_constraints), len(additional_constraints)]
        
        rhs = rhs_machine_capacity_processtime + rhs_preceding_transfer + rhs_continuous_processtime + rhs_adjacent_casts + self.duedate
        return rows, rhs, n_per_constraints
        
    def set_senses(self):
        s, senses = 'LLELE', ''
        for i in range(len(self.n_per_constraints)):
            senses += s[i] * self.n_per_constraints[i]
        return senses 
    
    def solve(self):
        self.List_Schedule = cplex.Cplex()
        self.List_Schedule.objective.set_sense(self.List_Schedule.objective.sense.minimize)
        
        ub = [cplex.infinity] * self.n_dim
        self.List_Schedule.variables.add(obj = self.compute_obj(), ub = ub, names = self.variable_names)
    
        rownames = ['c' + str(i)  for i in range(self.n_constraints)]

        self.List_Schedule.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = rownames)
        self.List_Schedule.solve()
        self.get_solvable_status()
        self.get_optvalues()
        
    def get_optvalues(self):
        self.obj_values = self.List_Schedule.solution.get_objective_value()
        print ("Solution value:   ",  self.obj_values)
        self.opt_solution = []
        for i, x in enumerate(self.List_Schedule.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
            
                
    def get_solvable_status(self):
        status = self.List_Schedule.solution.get_status()
        if status == self.List_Schedule.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.List_Schedule.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.List_Schedule.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
    
    def print_constraints_slack(self):
        slack = self.List_Schedule.solution.get_linear_slacks()
        for i in range(len(slack)):
            print("Row %d:  Slack = %10f" % (i, slack[i]))
            
    def write_model(self):
        self.List_Schedule.write('List_Schedule.lp')

In [5]:
class Dual_Problem():
    def __init__(self):
        self.lamd = np.array([0 for i in range((n_stage - 1) * n_job * n_job)])
        self.m, self.r, self.alpha = 5, 0.01, 1.0
        self.step_init, self.step, self.iteration_time = 0.0, 0.0, 0
        self.subgradients = np.array([0 for i in range((n_stage - 1) * n_job * n_job)])
        self.obj_values_iterations = []
        self.subgradients_norm_iterations = []
        self.step_iterations = []
        self.obj_values = 0
    
    def estimate_opt_obj_values(self, up_bound):
        self.compute_relaxed_problem_obj()
        self.estimation_opt_obj_values = (self.obj_values + up_bound) / 2.0
        
    def compute_stepsize_init(self):
        self.step_init = (self.estimation_opt_obj_values - self.obj_values) / np.linalg.norm(self.subgradients_init)**2
        
    def compute_relaxed_problem_obj(self):
        self.obj_values = 0
        t_coeff_w = subproblem_qp.compute_obj_wait_time()
        t_coeff_c = subproblem_qp.compute_obj_casting()
        t_coeff = [t_coeff_w[i] + t_coeff_c[i] for i in range(len(t_coeff_w))]
        for i in range(len(t_coeff)):
            self.obj_values += t_coeff[i] * subproblem_qp.opt_solution[i] 

        for i in range(n_job):
            for i1 in range(n_job): 
                if i != i1:
                    for j in range(n_stage - 1):
                        t1 = i + j * n_job
                        t2 = i1 + j * n_job
                        t5 = i + i1 * n_job + j * n_job * n_job
                        for k in range(n_machine_perstage):
                            for r in range(n_batch * pcast - 1):
                                t3 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1 
                                #print(t3, t4, subproblem_lp.opt_solution[t4], subproblem_lp.opt_solution[t3], subproblem_qp.opt_solution[t2], subproblem_qp.opt_solution[t1], processtime[t1])
                                #value.append(-self.lamd[t5] * subproblem_lp.opt_solution[t3] * subproblem_lp.opt_solution[t4] * (subproblem_qp.opt_solution[t2] - subproblem_qp.opt_solution[t1] - processtime[t1]))
                                self.obj_values -= self.lamd[t5] * subproblem_lp.opt_solution[t3] * subproblem_lp.opt_solution[t4] * (subproblem_qp.opt_solution[t2] - subproblem_qp.opt_solution[t1] - processtime[t1])   
        
        self.obj_values_iterations.append(self.obj_values)
        
    def is_obj_reduction(self):
        assert(len(self.obj_values_iterations) >= 2)
        if self.obj_values_iterations[-2] > self.obj_values_iterations[-1]:
            return True
        return False
    
    def compute_subgradients(self, xval, tval):
        self.subgradients = np.array([0 for i in range((n_stage - 1) * n_job * n_job)])
        for i in range(n_job):
            for i1 in range(n_job):
                if i != i1:
                    for j in range(n_stage - 1):
                        t1 = i + j * n_job
                        t2 = i1 + j * n_job
                        t5 = i + i1 * n_job + j * n_job * n_job
                        for r in range(n_batch * pcast - 1):
                            for k in range(n_machine_perstage):
                                t3 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1 
                                self.subgradients[t5] -= xval[t3] * xval[t4] * (tval[t2] - tval[t1] - processtime[t1])
                                #print(xval[t3], xval[t4], (tval[t2] - tval[t1] - processtime[t1]))
        if self.iteration_time == 0:
            self.subgradients_init = self.subgradients.copy()                    
    
    def compute_stepsize(self):
        if self.iteration_time == 0:
            self.step = self.step_init
        else:
            p = 1 - 1/(self.iteration_time ** self.r)
            self.alpha = self.alpha * (1 - 1/(self.m * self.iteration_time ** p))
            self.subgradients_norm = np.linalg.norm(self.subgradients)
            self.subgradients_norm_iterations.append(self.subgradients_norm)
            if self.subgradients_norm != 0:
                self.step = self.alpha * self.step_init * np.linalg.norm(self.subgradients_init) / self.subgradients_norm
            else:
                self.step = 0
                print("subgradients = 0")
        self.step_iterations.append(self.step)
        return self.step
    
    def update_lamd(self):
        if self.iteration_time == 0:
            self.lamd = self.lamd + self.step_init * self.subgradients_init
        else:
            self.lamd = self.lamd + self.step * self.subgradients
        self.lamd = np.maximum(self.lamd, 0)
        self.iteration_time += 1
        return self.lamd
    
    def plot_results(self, lower_bound_iterations):
        plt.figure()
        x = np.linspace(0, len(self.obj_values_iterations), len(self.obj_values_iterations))
        plt.plot(x, self.obj_values_iterations)
        plt.xlabel('iteration times')
        plt.ylabel('cost function of dual problem')
        plt.savefig("obj_values_iterations.jpg")
        plt.show()
        
        plt.figure()
        x = np.linspace(0, len(lower_bound_iterations), len(lower_bound_iterations))
        plt.plot(x, lower_bound_iterations)
        plt.xlabel('iteration times')
        plt.ylabel('lower_bound_iterations')
        plt.savefig("lower_bound_iterations.jpg")
        plt.show()
        
        plt.figure()
        x = np.linspace(0, len(self.subgradients_norm_iterations), len(self.subgradients_norm_iterations))
        plt.plot(x, self.subgradients_norm_iterations)
        plt.xlabel('iteration times')
        plt.ylabel('subgradients_norm')
        plt.savefig("subgradients_norm.jpg")
        plt.show()
        
        plt.figure()
        x = np.linspace(0, len(self.step_iterations), len(self.step_iterations))
        plt.plot(x, self.step_iterations)
        plt.xlabel('iteration times')
        plt.ylabel('stepsize')
        plt.savefig("stepsize.jpg")
        plt.show()
        
    def output_results(self, lower_bound_iterations, running_time):
        with open('lamd.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.lamd[i * n_job:(i + 1) * n_job]))
                f.write('\n')
        with open('subgradients.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.subgradients[i * n_job:(i + 1) * n_job]))
                f.write('\n')
        with open('lower_bound_iterations.txt', 'w') as f:
            for x in lower_bound_iterations:
                f.write(str(x))
                f.write('\n')
        with open('step_iterations.txt', 'w') as f:
            for x in self.step_iterations:
                f.write(str(x))
                f.write('\n')
        with open('running_time.txt', 'w') as f:
            f.write(str(running_time) +'s')
            f.write('\n')

In [6]:
def compute_heuristic_xval(tval):
    xval = [0 for i in range((n_stage - 1) * n_job * n_job)]
    j = 0
    while (j < (n_stage - 1)):
        date = []
        for i in range(j * n_job, (j + 1) * n_job):
            date.append((tval[i],i))
        result = sorted(date)
        for i in range(n_job):
            job_index = result[i][1] - j * n_job
            t1 = i
            k = t1 % n_machine_perstage
            r = 0    
            while (t1 > n_machine_perstage - 1):
                r += 1
                t1 -= n_machine_perstage
            xval[job_index * n_job + j * n_job * n_job + k * n_batch * pcast + r] = 1
            t1 +=1
        j += 1
    return xval

In [7]:
np.random.seed(0)
random.seed(0)
#n_stage, pcast, n_batch, n_machine_perstage = 3, 8, 5, 3
n_stage, pcast, n_batch, n_machine_perstage = 3, 5, 3, 3
#n_stage, pcast, n_batch, n_machine_perstage = 3, 1, 3, 2
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']
assert(len(stage_name) == n_stage)

print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of job per cast:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)

n_position = dict(zip(stage_name[0:n_stage - 1], [int(n_job / n_machine['steelmaking'])] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

processtime = [14 * random.random() + 36 for i in range(n_job * n_stage)]
transittime = [3 * random.random() + 3 for i in range(n_stage - 1)]
su = 80
duedate = [0 for i in range(pcast * n_machine['casting'])]

penalty_coeff = 10
d_coeff = {'d1': 10, 'd2': 110}
coeff = [10 + 20 * i for i in range(n_stage - 1)]
print(processtime)
print(transittime)
print(coeff)

Number of stage: 3
Cast number of per machine: 5
Number of job per cast: 3
Number of machine: {'steelmaking': 3, 'refining': 3, 'casting': 3}
Number of job: 45
n_position =  {'steelmaking': 15, 'refining': 15}
[47.82190592135068, 46.61136164116424, 41.88800213163183, 39.624834504101486, 43.15784609916052, 41.669077924305796, 46.97318024648682, 40.246378165104986, 42.672357358132984, 44.167348552370434, 48.71358039273469, 43.065615981443464, 39.945729821595855, 46.58125885820114, 44.657165953454644, 39.50708877907417, 48.73644758355536, 49.758996664527146, 47.343041303952255, 48.63032330615415, 40.34206597047066, 46.217644475641805, 48.58373603155191, 45.575775046816176, 42.60999801633799, 37.40981691295712, 42.078405696352974, 44.55241762821322, 48.78215474533057, 49.532489148790624, 42.67813687173804, 48.11433898880296, 39.64689234548743, 47.27038957818231, 43.68179025369825, 36.19658380229627, 46.07586560965536, 41.583529591139765, 47.54782968007526, 45.35414481724591, 36.01599947040

In [8]:
random.seed(0)
lamd = np.zeros((n_stage - 1) * n_job * n_job)
xval = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)] 
subproblem_qp = Subproblem_QP()
subproblem_qp.compute_obj_linear(xval, lamd)
subproblem_qp.compute_qmat(lamd)
subproblem_qp.solve()
x_init = compute_heuristic_xval(subproblem_qp.opt_solution)

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
QP Presolve eliminated 2 rows and 2 columns.
Aggregator did 75 substitutions.
Reduced QP has 70 rows, 88 columns, and 155 nonzeros.
Presolve time = 0.01 sec. (0.09 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =          1260.000000
Iteration:    44   Dual objective     =         86048.681978
Solution value:    86048.68197844276


In [9]:
list_schedule = List_Schedule(x_init)
list_schedule.solve()
t_init = list_schedule.opt_solution

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1 columns.
Aggregator did 33 substitutions.
Reduced LP has 198 rows, 131 columns, and 410 nonzeros.
Presolve time = 0.02 sec. (0.13 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =          1330.000000
Iteration:    62   Dual infeasibility =           260.000000
Iteration:    89   Dual objective     =         86163.417691
Solution value:    92151.36096865103


In [10]:
subproblem_lp = Subproblem_LP()
subproblem_lp.opt_solution = x_init.copy()
dual_problem = Dual_Problem()

In [ ]:
ts = time.clock()
maxiteration_times = 20
lower_bound_iterations = []
for iteration_times in range(maxiteration_times):
    dual_problem.compute_relaxed_problem_obj()
    print("low bound = ", dual_problem.obj_values, "upper bound = ", list_schedule.obj_values)
    
    dual_problem.compute_subgradients(subproblem_lp.opt_solution, subproblem_qp.opt_solution)
    
    if iteration_times == 0:
        dual_problem.estimate_opt_obj_values(list_schedule.obj_values)
        dual_problem.compute_stepsize_init()
        print(dual_problem.step_init)
    
    dual_problem.compute_stepsize()
    dual_problem.update_lamd() 
    
    while True:
        subproblem_lp.compute_obj(subproblem_qp.opt_solution, dual_problem.lamd)
        #subproblem_lp.solve()
    
        subproblem_qp.compute_obj_linear(subproblem_lp.opt_solution, dual_problem.lamd)
        subproblem_qp.compute_qmat(dual_problem.lamd)
        
        subproblem_lp.solve()
        subproblem_qp.solve()
        
        dual_problem.compute_relaxed_problem_obj()
        if dual_problem.is_obj_reduction():
            break
            
    lower_bound_iterations.append(dual_problem.obj_values)
    
running_time = time.clock() - ts
print("running time is", running_time) 

low bound =  86048.68197844291 upper bound =  92151.36096865103
0.021967412475641764
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.02 sec. (1.14 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =          -203.275951
Iteration:    96   Dual objective     =          -203.275951
Perturbation started.
Iteration:   101   Dual objective     =          -203.275951
Iteration:   173   Dual objective     =          -203.275947
Removing perturbation.

Dual simplex solved model.

Solution value:    -203.27595086864974
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 23
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            =

  11  -1.0970251e+08  -1.0970252e+08  1.34e-07  0.00e+00  7.47e-11
  12  -1.0970251e+08  -1.0970251e+08  3.65e-07  0.00e+00  6.42e-11
Barrier time = 0.03 sec. (0.92 ticks)

Total time on 8 threads = 0.03 sec. (0.92 ticks)
Solution value:    -109702513.17208123
low bound =  86066.010865412 upper bound =  92151.36096865103
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (1.14 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =       -711325.206506
Iteration:   102   Dual objective     =        -18557.312392
Iteration:   197   Dual objective     =        -18346.671499
Iteration:   286   Dual objective     =        -18321.750392
Iteration:   368   Dual objective     =        -18311.205767
Iteration:   448   Dual objective     =        -18255.457720

Dual simplex solved model.

Solution value:    -1825

   5  -3.2893595e+08  -3.2916071e+08  4.08e+00  0.00e+00  3.99e+02
   6  -3.2917842e+08  -3.2920992e+08  2.62e-01  0.00e+00  2.56e+01
   7  -3.2919387e+08  -3.2919883e+08  2.47e-02  0.00e+00  2.42e+00
   8  -3.2919544e+08  -3.2919782e+08  5.24e-09  0.00e+00  2.45e-10
   9  -3.2919668e+08  -3.2919695e+08  6.51e-09  0.00e+00  2.25e-10
  10  -3.2919682e+08  -3.2919685e+08  8.21e-08  0.00e+00  2.25e-10
  11  -3.2919684e+08  -3.2919684e+08  3.09e-07  0.00e+00  2.25e-10
Barrier time = 0.03 sec. (1.49 ticks)

Total time on 8 threads = 0.03 sec. (1.49 ticks)
Solution value:    -329196835.5989794
low bound =  86683.42503443168 upper bound =  92151.36096865103
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.02 sec. (1.14 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =      -1223199.796627
Iteration:   104   Dua

QP Presolve eliminated 3 rows and 3 columns.
QP Presolve added 0 rows and 90 columns.
Aggregator did 30 substitutions.
Reduced QP has 204 rows, 222 columns, and 1003 nonzeros.
Reduced QP objective Q matrix has 87 nonzeros.
Presolve time = 0.02 sec. (0.23 ticks)
Parallel mode: using up to 8 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 2529
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.23 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 8
  Rows in Factor            = 204
  Integer space required    = 984
  Total non-zeros in factor = 6755
  Total FP ops to factor    = 335113
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf          
   0   1.9311151e+06   1.7335042e+06  4.94e+04  0.00e+00  6.73e+06
   1  -3.0103294e+08  -9.0887294e+07  2.21e+04  0.00e+00  3.01e+06
   2  -4.5951505e+08  -3.6027257e+08  3.99e+03  0.00e+00  5.43e+05
   3  -4.7211550e+08  -4.5737580e+08  5.27e+0

Presolve time = 0.00 sec. (1.14 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =      -1598554.386343
Iteration:   103   Dual objective     =        -42060.328669
Iteration:   198   Dual objective     =        -41682.498116
Iteration:   289   Dual objective     =        -41676.056807
Iteration:   365   Dual objective     =        -41643.600057
Iteration:   448   Dual objective     =        -41631.347462
Iteration:   521   Dual objective     =        -41582.553432
Iteration:   589   Dual objective     =        -41564.809471

Dual simplex solved model.

Solution value:    -41564.80947070574
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 263
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.03 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 90
  Integer space required    = 215
  Total non-zeros in factor = 990
  Total FP ops to factor    = 14818
Tried aggreg

   2  -5.5576334e+08  -4.3444537e+08  4.08e+03  0.00e+00  6.62e+05
   3  -5.7154264e+08  -5.5403967e+08  5.17e+02  0.00e+00  8.38e+04
   4  -5.7282973e+08  -5.7281475e+08  2.77e+01  0.00e+00  4.49e+03
   5  -5.7365234e+08  -5.7385354e+08  1.45e+00  0.00e+00  2.34e+02
   6  -5.7382611e+08  -5.7386700e+08  2.32e-01  0.00e+00  3.77e+01
   7  -5.7383366e+08  -5.7386643e+08  6.61e-02  0.00e+00  1.07e+01
   8  -5.7385328e+08  -5.7385838e+08  9.21e-03  0.00e+00  1.49e+00
   9  -5.7385539e+08  -5.7385743e+08  5.92e-09  0.00e+00  3.98e-10
  10  -5.7385645e+08  -5.7385674e+08  7.63e-09  0.00e+00  4.68e-10
  11  -5.7385658e+08  -5.7385664e+08  7.16e-08  0.00e+00  4.12e-10
  12  -5.7385661e+08  -5.7385662e+08  2.75e-07  0.00e+00  4.33e-10
  13  -5.7385661e+08  -5.7385661e+08  8.64e-07  0.00e+00  4.24e-10
Barrier time = 0.03 sec. (1.94 ticks)

Total time on 8 threads = 0.03 sec. (1.94 ticks)
Solution value:    -573856613.8692492
CPXPARAM_Read_DataCheck                          1
Parallel mode: dete

In [ ]:
subproblem_lp.output_results()
subproblem_qp.output_results()
subproblem_lp.write_model()
subproblem_qp.write_model()
dual_problem.plot_results(lower_bound_iterations)
dual_problem.output_results(lower_bound_iterations, running_time)

In [ ]:
n_position_permachine = int(n_job / n_machine_perstage)
x_opt = np.array(subproblem_lp.opt_solution).reshape(((n_stage - 1), n_job, n_position_permachine, n_machine_perstage))
t_opt = np.array(subproblem_qp.opt_solution[0: n_stage * n_job]).reshape((n_stage, n_job))

In [ ]:
x_opt_index = np.where(x_opt == 1)
print(t_opt[0,:])
x_opt_index_list = list(zip(x_opt_index[0], x_opt_index[1], x_opt_index[2], x_opt_index[3]))
print(x_opt_index_list)
print(len(x_opt_index_list))